In [1]:
include("clustering.jl");
using .ShipClustering

using CSV
using DataFrames
using Statistics
using Random
using PlotlyJS

## Какая проблема?
Защитить свой шаттл в результате борьбы.

Но я не оптимизирую защиту. Только нападение.

## Оптимальное решение

Мы будем искать кластеры по окружностям, так как считаем, что наше оружие эффективно только в определенном радиусе.  
То есть задача найти такие цели, в радиус которых попадает как можно больше вражеских кораблей.

In [2]:
csv = CSV.File(joinpath("data", "Ship game.csv"); header=1, skipto=4, delim=";", select=1:4);
csv4d = CSV.File(joinpath("data", "Ship game 4d.csv"); header=1, skipto=4, delim=";", select=1:5);

Наши предположения:
1. Оружие поражает всех в радиусе `RADIUS_DAMAGE`
2. За одно мгновение можно поразить `NUM_TARGETS`
3. Само положение шаттла или корабля игнорируется
4. Качество падания и радиус действия не изменяется в расстоянием.

Критерий качества кластеризации: максимальное кол-во противников, которые могут быть поражены с учетом вышеописанных констант.

Получение датафреймов (3d+4d)

In [3]:
df = DataFrame(csv)
df = rename(df, ["ship", "x", "y", "z"])

Row,ship,x,y,z
,String7,Int64,Int64,Int64
1,CSM_1,200,150,150
2,CSM_2,210,130,140
3,CSM_3,205,120,135
4,CSM_4,205,130,135
5,CSM_5,205,50,50
6,CSM_6,205,65,50
7,CSM_7,210,50,70
8,CSM_8,200,30,50
9,CSM_9,100,45,50


In [4]:
df4d = DataFrame(csv4d, [:ship, :x, :y, :z, :d])
# масштабирование колонки, чтобы она несла одинаковую нагрузку, как и остальные колонки
df4d[!, :d] = df4d.d / 12 * 220
df4d

Row,ship,x,y,z,d
,String7,Int64,Int64,Int64,Float64
1,CSM_1,200,150,150,73.3333
2,CSM_2,210,130,140,165.0
3,CSM_3,205,120,135,128.333
4,CSM_4,205,130,135,146.667
5,CSM_5,205,50,50,128.333
6,CSM_6,205,65,50,128.333
7,CSM_7,210,50,70,55.0
8,CSM_8,200,30,50,36.6667
9,CSM_9,100,45,50,128.333


Перевод датафреймов в массив, с которым будем работать

In [5]:
coords = Matrix(df[:, 2:end])

10×3 Matrix{Int64}:
 200  150  150
 210  130  140
 205  120  135
 205  130  135
 205   50   50
 205   65   50
 210   50   70
 200   30   50
 100   45   50
  45  100   92

In [6]:
coords4d = Matrix(df4d[:, 2:end])

13×4 Matrix{Float64}:
 200.0  150.0  150.0   73.3333
 210.0  130.0  140.0  165.0
 205.0  120.0  135.0  128.333
 205.0  130.0  135.0  146.667
 205.0   50.0   50.0  128.333
 205.0   65.0   50.0  128.333
 210.0   50.0   70.0   55.0
 200.0   30.0   50.0   36.6667
 100.0   45.0   50.0  128.333
  45.0  100.0   92.0   73.3333
 150.0   90.0   60.0   73.3333
  90.0  102.0   54.0  165.0
 140.0   87.0   89.0   18.3333

Алгоритм кластеризации:
1. Начинаем со случайной точки. Она в кластере 1.
2. Находим у текущего кластера центроиду.
3. Находим ближайшую свободную точку к центроиде.
4. Находим новую центроиду после добавления новой точки к кластеру.
   1. Если новая точка дальше, чем наш порог (радиус поражения) или кластер с новой центроидой теряет старые точки, то увеличиваем id кластера и переходим к шагу 2.
   2. Иначе добавляем точку в текущий кластер и переходим к шагу 2.

Пример одной жадной кластеризации

In [7]:
clusters = find_clusters_greedy(coords, 20)

Dict{Int64, Vector{Int64}} with 7 entries:
  5 => [10]
  4 => [7]
  6 => [8]
  7 => [9]
  2 => [3, 4, 2]
  3 => [1]
  1 => [5, 6]

Поиск лучшей кластеризации через симуляцию нескольких экспериментов. Вывод отсортирован по пользе кластеров.

In [8]:
find_clusters(coords, 20)

7-element Vector{Vector{Int64}}:
 [3, 4, 2]
 [6, 5]
 [7]
 [10]
 [9]
 [1]
 [8]

## Plots

Получение результата в датафрейме, где указаны координаты центроид лучших кластеров, и какие корабли будут затронуты

In [9]:
clusters_constrained = get_clusters_with_constraints(coords)
centers_targets = DataFrame(get_targets(coords, clusters_constrained), [:x, :y, :z])
centers_targets[!, :targeted] = clusters_constrained
centers_targets

Row,x,y,z,targeted
,Float64,Float64,Float64,Array…
1,206.667,126.667,136.667,"[3, 4, 2]"
2,205.0,57.5,50.0,"[6, 5]"


In [10]:
function plot_clustering_3d(radius_damage)
    clusters_constrained = get_clusters_with_constraints(coords, radius_damage=radius_damage)
    centers_targets = DataFrame(get_targets(coords, clusters_constrained), [:x, :y, :z])
    centers_targets[!, :targeted] = clusters_constrained

    targeted = Set(reduce(vcat, clusters_constrained))
    color_targeted = [i in targeted ? "red" : "blue" for i in 1:size(df)[1]]
    df[!, :color_targeted] = color_targeted
    trace_dots = scatter3d(
        df,
        x=:x,
        y=:y,
        z=:z,
        text=:ship,
        mode="markers",
        marker=attr(
            size=:5,
            color=:color_targeted,
            sizemode="diameter"
        ),
        showlegend=false,
    )
    trace_bubble = scatter3d(
        centers_targets,
        x=:x,
        y=:y,
        z=:z,
        mode="markers",
        marker=attr(
            size=radius_damage * 2,
            color="red",
            sizemode="diameter",
            opacity=0.3,
        ),
        showlegend=false,
    )

    layout = Layout(scene_zaxis_type="log", title="Лучшие цели. Radius=$radius_damage")
    plot([trace_dots, trace_bubble], layout)
end

plot_clustering_3d (generic function with 1 method)

In [11]:
# plot_clustering_3d(10) # для интерактивного графика

Почему-то здесь julia любит зависать, поэтому закомментировал.

In [12]:
# for radius in [1, 10, 20, 50, 100]
#     plot_targets = plot_clustering_3d(radius)
#     savefig(plot_targets, "imgs/targets_3d_$radius.png", height=500, width=800, scale=2)
# end

## 4D

Работа с 4 измерениями абсолютно аналогичная. Берем во внимание, что у 4 измерения была меньшая размерность.  
 Мы её вытянули как у первых трех измерений, потом обратно сжали. Так мы работали с измерениями, которые вносят равный вклад в кластеризацию, даже если у них разная размерность.

Нарисовать уже не получится

In [13]:
clusters_constrained = get_clusters_with_constraints(coords4d, radius_damage=50)
centers_targets = DataFrame(get_targets(coords4d, clusters_constrained), [:x, :y, :z, :d])
centers_targets[!, :targeted] = clusters_constrained
# восстановим значение в прежний масштаб
centers_targets[!, :d] = centers_targets.d / 220 * 12
centers_targets

Row,x,y,z,d,targeted
,Float64,Float64,Float64,Float64,Array…
1,206.667,126.667,136.667,8.0,"[3, 4, 2]"
2,205.0,57.5,50.0,7.0,"[5, 6]"


## Зависимость качества от радиуса

Мы фиксируем максимальное кол-во целей - 2 и варируем радиус действия.

Оценка - кол-во вражеских кораблей, которые можно поразить в один момент с указанным или большим радиусом.

In [14]:
best_score = 0
for radius in 1:1:300
    l = Dict([Pair(num, i) for (num, i) in enumerate(get_clusters_with_constraints(coords4d; radius_damage=radius))])
    score = score_clustering(coords4d, l, 2)[1]
    if score > best_score
        best_score = score
        println("radius: $radius, best score: $best_score")
    end
end

radius: 1, best score: 2
radius: 15, best score: 3
radius: 20, best score: 4
radius: 29, best score: 5


radius: 65, best score: 6
radius: 77, best score: 9
radius: 90, best score: 10
radius: 104, best score: 11
radius: 110, best score: 12
radius: 113, best score: 13
